# Exercise 1

# Exercise 2

# Exercise 3

# Exercise 4: The Miller-Rabin Primality Test

Exercise 4 asks for a Python program based on the strategy described in the provided text. This strategy is the **Miller-Rabin primality test**, which is a probabilistic test utilized to efficiently determine if a large number $n$ is prime.

The algorithm relies on the logic established in Theorems 1 and 2 of the text. The process is as follows:

1.  **Decomposition**:
    First, we take the number $n-1$ (where $n$ is the number we want to test) and write it in the form $n-1 = 2^k d$, where $d$ is an odd number.

2.  **Witness Selection**:
    We pick a random integer $a$ (the "witness") such that $1 < a < n-1$.

3.  **Test Conditions**:
    According to Theorem 2, if $n$ is prime, then one of the following conditions must be true:
    * $a^d \equiv 1 \pmod{n}$
    * $a^{2^i d} \equiv -1 \pmod{n}$ for some $i$ in the range $0 \le i \le k-1$.

4.  **Conclusion**:
    * If $a$ satisfies **either** of these conditions, it is not a witness to $n$'s compositeness, and $n$ might be prime.
    * If $a$ satisfies **neither** condition, it *is* a witness. We have definitively proven that $n$ is composite.

5.  **Iteration**:
    We repeat this test $m$ times with $m$ different random witnesses. If $n$ passes all $m$ tests, we can be confident it is prime with a probability of $1 - 2^{-m}$.

A python implementation of this test can be found at **miller-rabbin.py**.

# Exercise 5